In [7]:
import numpy as np
import cv2
import glob
import os
import json

# termination criteria
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)

# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((7 * 7, 3), np.float32)
objp[:, :2] = np.mgrid[0:7, 0:7].T.reshape(-1, 2)

# Arrays to store object points and image points from all the images.
objpoints = []  # 3d point in real world space
imgpoints = []  # 2d points in image plane.

path = './Imagenes_CEL/'
#path_file = os.path.join(path, 'left*.jpg')
path_file = './Imagenes_CEL/left*.jpg'
images = glob.glob(path_file)

for fname in images:
    img = cv2.imread(fname)
    
    WIDTH = 960
    HEIGHT = 720
    img = cv2.resize(img, (WIDTH,HEIGHT), interpolation=cv2.INTER_CUBIC)
    
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    cv2.imshow('img', gray)
    cv2.waitKey(250)

    # Find the chess board corners
    ret, corners = cv2.findChessboardCorners(gray, (7, 7), None)

    # If found, add object points, image points (after refining them)
    if ret == True:
        objpoints.append(objp)

        corners2 = cv2.cornerSubPix(gray, corners, (11, 11), (-1, -1), criteria)
        imgpoints.append(corners2)

        # Draw and display the corners
        img = cv2.drawChessboardCorners(img, (7, 7), corners2, ret)
        cv2.imshow('img', img)
        cv2.waitKey(250)

cv2.destroyAllWindows()



ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)
d = 3
h = 2
tilt = 30
pan = 0

file_name = 'calibration.json'
json_file = os.path.join(path, file_name)

data = {
    'K': mtx.tolist(),
    'd': d,
    'h': h,
    'tilt': tilt,
    'pan': pan
    #'distortion': dist.tolist()
}

with open(json_file, 'w') as fp:
    json.dump(data, fp, sort_keys=True, indent=1, ensure_ascii=False)

with open(json_file) as fp:
    json_data = json.load(fp)
print(json_data)


{'K': [[770.8314262922523, 0.0, 483.0177694648179], [0.0, 773.873653043318, 353.15583669349905], [0.0, 0.0, 1.0]], 'd': 3, 'h': 2, 'pan': 0, 'tilt': 30}
